In [3]:
import pandas as pd
import numpy as np

get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt
from matplotlib import gridspec
from dataload import dataload
from ploting import ploting,curve_fit_ploting ,auto_arima_ploting,DNN_ploting
import numpy as np
from sklearn.datasets import load_linnerud
from sklearn.linear_model import Ridge
from pmdarima.arima import ndiffs
from scipy.optimize import curve_fit
from curve_fit_func import func
from fitting import fitting_result
from pmdarima.arima import ndiffs
import pmdarima as pm
import time
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from datetime import datetime,timedelta
import os
# from torchinfo import summary
# import torchvision 
from sklearn.preprocessing import StandardScaler
from curve_ptf import curve_ptf
from aroma_ptf import arima_ptf
from DNN_ptf import DNN_ptf



In [5]:
expected_len = 120  
# 학습구간 + pred_len
pred_len_list = [7,10,15,20,25]
pred_window = 30 # 예측 구간 길이
weight = [0.1,0.8,0.1] # 탐지 웨이트 
df, _, ne_ru_list=dataload() # 
count_dict=pd.DataFrame([np.zeros(len(pred_len_list))],columns=pred_len_list,index=ne_ru_list) 


In [6]:
###################################### Start  ###############################################################
for ru in ne_ru_list:
    for pred_len in pred_len_list:

        ru=ru                                                                   #'LGE0013403_6A' #ru
        print(ru,pred_len)
        feature_list            = ['DU_TX' , 'DU_RX' , 'RU_TX' , 'RU_RX']

        df_tmp                  = df[df.RU_Name == ru].reset_index(drop =True)
        df_tmp                  = df_tmp.dropna(subset=feature_list)
        df_tmp                  = df_tmp.fillna("NAN")

        fault_date              = df_tmp[df_tmp['점검']!= 'NAN']['datetime'].unique()[0]
        input_x_or              = df_tmp.loc[df_tmp['datetime']<=fault_date,feature_list] 
        length                  = int(np.round(np.shape(input_x_or.values)[0])) #고장시점으로부터 데이터 첫 수집 데이터날 
        ##########################최소 데이터 ###############################
        if length <= 70:
        #             print("break")
            break

        curve_fit_result,_,_,_,_ = curve_ptf(ru=ru,df=df, expected_len=expected_len,pred_len=pred_len,input_x_or=input_x_or, pred_window=pred_window )
        arima_result = arima_ptf(ru=ru,df=df, expected_len=expected_len,pred_len=pred_len, pred_window=pred_window,input_x_or=input_x_or )
        DNN_result = DNN_ptf(ru=ru,df=df, expected_len=expected_len,pred_len=pred_len, pred_window=pred_window,input_x_or=input_x_or,ne_ru_list=ne_ru_list )


#         count_dict[pred_len,ru] = count_dict[pred_len]+fitting_result(curve_fit_result=curve_fit_result,arima_result=arima_result,DNN_result=DNN_result,weight=weight)
        count_dict.loc[ru,pred_len] = count_dict.loc[ru,pred_len]+fitting_result(curve_fit_result=[],arima_result=arima_result,DNN_result=[],weight=weight)
        # 안쓰는 피팅 모델은 [] 로 넣어주시면 됩니다.

LGE0013407_6A 7
LGE0013843_5B 7
LGE0013202_1A 7
LGE0014496_2C 7
LGE0011553_6 7
LGE0011380_5 7
LGE0011364_6 7
LGE0013384_4A 7
LGE0014526_7 7
LGE0011575_1 7
LGE0013843_5A 7
LGE0013247_4A 7
LGE0012782_6 7
LGE0011603_3 7
LGE0013491_4B 7
LGE0012913_2C 7
LGE0012958_1B 7
LGE0011322_2 7
LGE0013833_1B 7
LGE0012880_4B 7
LGE0013646_12 7
LGE0014554_5B 7
LGE0013843_9 7
LGE0014614_8 7
LGE0014293_3B 7
LGE0014293_3A 7
LGE0011395_10 7
LGE0011649_2 7
LGE0014476_12 7
LGE0013066_4B 7
LGE0014359_1 7
LGE0011395_8 7
LGE0013131_4B 7
LGE0014577_4 7
LGE0011359_6A 7
LGE0014550_6 7
LGE0013174_11 7
LGE0011292_8 7
LGE0014556_12N 7
LGE0014559_1 7
LGE0012792_2A 7
LGE0012521_1C 7


/Users/hjl/Documents/Optical Fault Checker/Optic Line Fault Checker/curve_ptf.py:32: RuntimeWarning: overflow encountered in exp
  return -1 * np.exp((1/b ) * (x-a)) + c +d*x
/Users/hjl/date/envs/torch/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/Users/hjl/Documents/Optical Fault Checker/Optic Line Fault Checker/curve_ptf.py:32: RuntimeWarning: overflow encountered in exp
  return -1 * np.exp((1/b ) * (x-a)) + c +d*x
/Users/hjl/Documents/Optical Fault Checker/Optic Line Fault Checker/curve_ptf.py:32: RuntimeWarning: overflow encountered in exp
  return -1 * np.exp((1/b ) * (x-a)) + c +d*x


확인 구간: 73
학습 길이 기준 미달
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-306.199, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-334.336, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-348.685, Time=0.01 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-308.199, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-351.021, Time=0.02 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-347.291, Time=0.03 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-348.472, Time=0.04 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-351.491, Time=0.02 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=-346.304, Time=0.08 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=-347.577, Time=0.06 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=-353.586, Time=0.03 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=-352.062, Time=0.02 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=-351.601, Time=0.05 sec
 ARIMA(0,1,3)(0,0,0)[0]             : AIC=-351.595, Time=0.05 sec
 ARIMA(1,1,

/Users/hjl/Documents/Optical Fault Checker/Optic Line Fault Checker/DNN_ptf.py:224: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/runner/miniforge3/conda-bld/pytorch-recipe_1658221004110/work/torch/csrc/utils/tensor_new.cpp:204.)
  input_x = torch.tensor([input_x],dtype=torch.float32).to(device)
/Users/hjl/Documents/Optical Fault Checker/Optic Line Fault Checker/curve_ptf.py:32: RuntimeWarning: overflow encountered in exp
  return -1 * np.exp((1/b ) * (x-a)) + c +d*x
/Users/hjl/date/envs/torch/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/Users/hjl/Documents/Optical Fault Checker/Optic Line Fault Checker/curve_ptf.py:32: RuntimeWarning: overflow encounter

total_result : 0.0
No Problem
LGE0012521_1C 10
확인 구간: 73
학습 길이 기준 미달
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-292.519, Time=0.00 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-321.418, Time=0.01 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-334.347, Time=0.02 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-294.519, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-337.470, Time=0.05 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-333.509, Time=0.02 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-335.522, Time=0.06 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-337.270, Time=0.07 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=-327.356, Time=0.02 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-333.470, Time=0.07 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=-339.353, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=-337.469, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=-323.412, Time=0.01 sec
 ARIMA(2,1,1)(0,0,0)[0]       

...

 ARIMA(3,0,2)(0,0,0)[0]             : AIC=-502.352, Time=0.13 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=inf, Time=0.08 sec
 ARIMA(3,0,1)(0,0,0)[0] intercept   : AIC=-528.273, Time=0.05 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=-530.226, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=-531.445, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=-532.609, Time=0.01 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=-533.041, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=-533.202, Time=0.02 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=-532.322, Time=0.02 sec

Best model:  ARIMA(1,0,0)(0,0,0)[0] intercept
Total fit time: 0.518 seconds
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-163.284, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-162.250, Time=0.01 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-163.564, Time=0.02 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-163.055, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] int

/Users/hjl/date/envs/torch/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.06 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0] intercept
Total fit time: 0.103 seconds
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-456.745, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-470.041, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.04 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-458.745, Time=0.02 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=-487.087, Time=0.04 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=-491.228, Time=0.03 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=-494.444, Time=0.09 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-496.134, Time=0.05 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-495.238, Time=0.07 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-494.485, Time=0.07 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.08 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=-494.440, Time=0.10 sec
 ARIMA(2,1,1)(0,0,0)[0]             :

/Users/hjl/date/envs/torch/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


 ARIMA(3,0,2)(0,0,0)[0]             : AIC=-427.047, Time=0.13 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=inf, Time=0.09 sec
 ARIMA(3,0,1)(0,0,0)[0] intercept   : AIC=-453.463, Time=0.03 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=-455.417, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=-456.973, Time=0.03 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=-458.729, Time=0.02 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=-460.207, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=-458.840, Time=0.01 sec

Best model:  ARIMA(0,0,0)(0,0,0)[0] intercept
Total fit time: 0.492 seconds
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-137.220, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-135.972, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-136.900, Time=0.02 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-137.126, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.05 sec

Best model:  ARIMA(0,1,0)(0,0,0

In [8]:
count_dict

,7,10,15,20,25
LGE0013407_6A,0.0,0.0,0.0,0.0,0.0
LGE0013843_5B,0.0,0.0,0.0,0.0,0.0
LGE0013202_1A,0.0,0.0,0.0,0.0,0.0
LGE0014496_2C,0.0,0.0,0.0,0.0,0.0
LGE0011553_6,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
LGE0012883_6A,0.0,0.0,0.0,0.0,0.0
LGE0013242_4,0.0,0.0,0.0,0.0,0.0
LGE0013249_1A,0.0,0.0,0.0,0.0,0.0
LGE0014550_5,0.0,0.0,0.0,0.0,0.0


'LGE0014411_5B'

In [6]:
len(count_dict.sum(axis=1)[count_dict.sum(axis=1)>=1])

19

In [7]:
19

19